# Library Imports and Settings

In [385]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
%pip install missingno
import missingno as msno
from datetime import date
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler
from scipy.stats import norm
from sklearn.impute import KNNImputer
import seaborn as sns

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [386]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)

# Method Definitions 

In [387]:
#FOR DATASET LOADING
def load_train():
    data = pd.read_csv("../HPP/train.csv")
    return data

def load_test():
    data = pd.read_csv("../HPP/test.csv")
    return data

def concat_df_on_y_axis(df_1, df_2):
    return pd.concat([df_1, df_2])

def concat_df_on_x_axis(df_1, df_2):
    return pd.concat([df_1, df_2], axis = 1)

def grab_col_names(dataframe, cat_th=10, car_th=20):
    """

    Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
    Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.

    Parameters
    ------
        dataframe: dataframe
                Değişken isimleri alınmak istenilen dataframe
        cat_th: int, optional
                numerik fakat kategorik olan değişkenler için sınıf eşik değeri
        car_th: int, optinal
                kategorik fakat kardinal değişkenler için sınıf eşik değeri

    Returns
    ------
        cat_cols: list
                Kategorik değişken listesi
        num_cols: list
                Numerik değişken listesi
        cat_but_car: list
                Kategorik görünümlü kardinal değişken listesi

    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grab_col_names(df))


    Notes
    ------
        cat_cols + num_cols + cat_but_car = toplam değişken sayısı
        num_but_cat cat_cols'un içerisinde.
        Return olan 3 liste toplamı toplam değişken sayısına eşittir: cat_cols + num_cols + cat_but_car = değişken sayısı

    """

    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car


#FOR OUTLIERS
def outlier_thresholds(dataframe, col_name, q1=0.05, q3=0.95):
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def check_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False


def grab_outliers(dataframe, col_name, index=False):
    low, up = outlier_thresholds(dataframe, col_name)

    if dataframe[((dataframe[col_name] < low) | (dataframe[col_name] > up))].shape[0] > 10:
        print(dataframe[((dataframe[col_name] < low) | (dataframe[col_name] > up))].head())
    else:
        print(dataframe[((dataframe[col_name] < low) | (dataframe[col_name] > up))])

    if index:
        outlier_index = dataframe[((dataframe[col_name] < low) | (dataframe[col_name] > up))].index
        return outlier_index

def remove_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    df_without_outliers = dataframe[~((dataframe[col_name] < low_limit) | (dataframe[col_name] > up_limit))]
    return df_without_outliers

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

# FOR MISSING VALUES
def missing_values_table(dataframe, na_name=False):
    na_columns = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]

    n_miss = dataframe[na_columns].isnull().sum().sort_values(ascending=False)
    ratio = (dataframe[na_columns].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])
    print(missing_df, end="\n")

    if na_name:
        return na_columns


#FOR ENCODERS
def one_hot_encoder(dataframe, categorical_cols, drop_first=True):
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=drop_first)
    return dataframe

def one_hot_encoder_na_dummy(dataframe, categorical_cols, drop_first=True, dummy_na=True):
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=drop_first, dummy_na = dummy_na)
    return dataframe

def rare_analyser(dataframe, target, cat_cols):
    for col in cat_cols:
        print(col, ":", len(dataframe[col].value_counts()))
        print(pd.DataFrame({"COUNT": dataframe[col].value_counts(),
                            "RATIO": dataframe[col].value_counts() / len(dataframe),
                            "TARGET_MEAN": dataframe.groupby(col)[target].mean()}), end="\n\n\n")

def rare_analyser(dataframe, cat_cols):
    for col in cat_cols:
        print(col, ":", len(dataframe[col].value_counts()))
        print(pd.DataFrame({"COUNT": dataframe[col].value_counts(),
                            "RATIO": dataframe[col].value_counts() / len(dataframe)}), end="\n\n\n")

def rare_encoder(dataframe, rare_perc):
    temp_df = dataframe.copy()

    rare_columns = [col for col in temp_df.columns if temp_df[col].dtypes == 'O'
                    and (temp_df[col].value_counts() / len(temp_df) < rare_perc).any(axis=None)]

    for var in rare_columns:
        tmp = temp_df[var].value_counts() / len(temp_df)
        rare_labels = tmp[tmp < rare_perc].index
        temp_df[var] = np.where(temp_df[var].isin(rare_labels), 'Rare', temp_df[var])

    return temp_df

#FOR CORREALTION ANALYSIS
def high_correlated_cols(dataframe, plot=False, corr_th=0.70):
    corr = dataframe.corr()
    cor_matrix = corr.abs()
    upper_triangle_matrix = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k=1).astype(np.bool))
    drop_list = [col for col in upper_triangle_matrix.columns if any(upper_triangle_matrix[col] > corr_th)]
    if plot:
        #import seaborn as sns
        #import matplotlib.pyplot as plt
        sns.set(rc={'figure.figsize': (15, 15)})
        sns.heatmap(corr, cmap="RdBu")
        plt.show()
    return drop_list

# Preprocessing

## Dataset Reading

In [388]:
df_train = load_train()
df_test = load_test()
df = concat_df_on_y_axis(df_train.drop(['SalePrice'], axis = 1), df_test)
#df = df.drop['SalePrice']
df.shape

(2919, 80)

In [389]:
df_copy = df.copy()
df_copy.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.000,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.000,Gd,TA,PConc,Gd,TA,No,GLQ,706.000,Unf,0.000,150.000,856.000,GasA,Ex,Y,SBrkr,856,854,0,1710,1.000,0.000,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.000,RFn,2.000,548.000,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.000,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.000,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.000,Unf,0.000,284.000,1262.000,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.000,1.000,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.000,RFn,2.000,460.000,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.000,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.000,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.000,Unf,0.000,434.000,920.000,GasA,Ex,Y,SBrkr,920,866,0,1786,1.000,0.000,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.000,RFn,2.000,608.000,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.000,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.000,TA,TA,BrkTil,TA,Gd,No,ALQ,216.000,Unf,0.000,540.000,756.000,GasA,Gd,Y,SBrkr,961,756,0,1717,1.000,0.000,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.000,Unf,3.000,642.000,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.000,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.000,Gd,TA,PConc,Gd,TA,Av,GLQ,655.000,Unf,0.000,490.000,1145.000,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.000,0.000,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.000,RFn,3.000,836.000,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [390]:
df_copy.drop(['Id'], axis = 1, inplace = True)
print(grab_col_names(df_copy))

Observations: 2919
Variables: 79
cat_cols: 52
num_cols: 26
cat_but_car: 1
num_but_cat: 10
(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition', 'OverallCond', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'Fireplaces', 'GarageCars', 'YrSold'], ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivAr

## Grabbing Columns (NUM - CAT)

In [391]:
cols_with_na_meaning = ['Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 
                            'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']

In [392]:
for col in cols_with_na_meaning:
    df_copy[col].fillna("no_feature",inplace=True)

In [393]:
encoding = {
    'no_feature': 0,
    'Po': 1, 'No': 1, 'Unf': 1, 'Sal': 1, 'MnWw': 1,
    'Fa': 2, 'Mn': 2, 'LwQ': 2, 'Sev': 2, 'RFn': 2, 'GdWo': 2,
    'TA': 3, 'Av': 3, 'Rec': 3, 'Maj2': 3, 'Fin': 3, 'MnPrv': 3,
    'Gd': 4, 'BLQ': 4, 'Maj1': 4, 'GdPrv': 4,
    'Ex': 5, 'ALQ': 5, 'Mod': 5,
    'GLQ': 6, 'Min2': 6,
    'Min1': 7,
    'Typ': 8,
}

# Kodlamayı uygulayacağımız sütunlar listesi
columns_to_encode = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 
                     'KitchenQual', 'Functional', 'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence']

# Kodlamayı uygulama
for column in columns_to_encode:
    df_copy[column] = df_copy[column].map(encoding)

In [394]:
cat_cols, num_cols, cat_but_car = grab_col_names(df_copy)

Observations: 2919
Variables: 79
cat_cols: 52
num_cols: 26
cat_but_car: 1
num_but_cat: 26


In [395]:
print(cat_cols)
print('-----------------------')
print(num_cols)
print('-----------------------')
print(cat_but_car)

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'CentralAir', 'Electrical', 'GarageType', 'PavedDrive', 'MiscFeature', 'SaleType', 'SaleCondition', 'OverallCond', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageFinish', 'GarageCars', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'YrSold']
-----------------------
['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'TotRmsAbvGrd', 'GarageYrBlt', 'GarageArea', 'WoodDeckSF', 'Ope

In [396]:
known_num_cols = ['BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'Fireplaces', 'GarageCars', 'OverallCond', 'YrSold']

In [397]:
for col in known_num_cols:
    if num_cols.__contains__(col) == False:
        num_cols.append(col)
        cat_cols.remove(col)

for col in columns_to_encode:
    if num_cols.__contains__(col) == False:
        num_cols.append(col)
        cat_cols.remove(col)

In [398]:
print(cat_cols)
print('-----------------------')
print(num_cols)
print('-----------------------')
print(cat_but_car)

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'CentralAir', 'Electrical', 'GarageType', 'PavedDrive', 'MiscFeature', 'SaleType', 'SaleCondition']
-----------------------
['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'TotRmsAbvGrd', 'GarageYrBlt', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'Fireplaces', 'GarageCars', 'OverallCond', 'YrSold', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'KitchenQual', 'Fu

## Suppressing Outliers of Numeric Columns

In [399]:
print(len(num_cols))

print(len(cat_cols))

print('*------------------------*')

for col in num_cols:
    print(col, check_outlier(df_copy, col))

for col in num_cols:
    replace_with_thresholds(df_copy, col)

print('------------------------')

for col in num_cols:
    print(col, check_outlier(df_copy, col))

print('*------------------------*')

52
26
*------------------------*
MSSubClass False
LotFrontage True
LotArea True
OverallQual False
YearBuilt False
YearRemodAdd False
MasVnrArea True
BsmtFinSF1 True
BsmtFinSF2 True
BsmtUnfSF False
TotalBsmtSF True
1stFlrSF True
2ndFlrSF False
LowQualFinSF True
GrLivArea True
TotRmsAbvGrd False
GarageYrBlt True
GarageArea False
WoodDeckSF True
OpenPorchSF True
EnclosedPorch True
3SsnPorch True
ScreenPorch True
PoolArea True
MiscVal True
MoSold False
BsmtFullBath True
BsmtHalfBath False
FullBath True
HalfBath False
BedroomAbvGr True
KitchenAbvGr True
Fireplaces False
GarageCars False
OverallCond False
YrSold False
ExterQual False
ExterCond True
BsmtQual False
BsmtCond True
BsmtExposure False
BsmtFinType1 False
BsmtFinType2 False
HeatingQC False
KitchenQual False
Functional True
FireplaceQu False
GarageFinish False
GarageQual False
GarageCond False
PoolQC True
Fence False
------------------------
MSSubClass False
LotFrontage False
LotArea False
OverallQual False
YearBuilt False
YearRemodA

C:\Users\BORA\AppData\Local\Temp\ipykernel_12188\574252897.py:113: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-17759.35' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
C:\Users\BORA\AppData\Local\Temp\ipykernel_12188\574252897.py:113: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1864.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
C:\Users\BORA\AppData\Local\Temp\ipykernel_12188\574252897.py:113: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-1080.3999999999996' has dtype incompatible with int64, please explicitly cast to a compatible dtype first

GarageArea False
WoodDeckSF False
OpenPorchSF False
EnclosedPorch False
3SsnPorch False
ScreenPorch False
PoolArea False
MiscVal False
MoSold False
BsmtFullBath False
BsmtHalfBath False
FullBath False
HalfBath False
BedroomAbvGr False
KitchenAbvGr False
Fireplaces False
GarageCars False
OverallCond False
YrSold False
ExterQual False
ExterCond False
BsmtQual False
BsmtCond False
BsmtExposure False
BsmtFinType1 False
BsmtFinType2 False
HeatingQC False
KitchenQual False
Functional False
FireplaceQu False
GarageFinish False
GarageQual False
GarageCond False
PoolQC False
Fence False
*------------------------*


## Categorization of Year Columns

In [400]:
year_cols = ['YearRemodAdd', 'GarageYrBlt', 'YearBuilt', 'YrSold']

# Sınıf aralıklarını ve etiketlerini tanımlama
bins = [1871, 1900, 1945, 1970, 1990, 2000, 2010]
labels = ['1872-1900', '1901-1945', '1946-1970', '1971-1990', '1991-2000', '2001-2010']

for col in year_cols:
    df_copy[col] = pd.cut(df[col], bins=bins, labels=labels, right=True)


In [401]:
cat_cols.extend(year_cols)
num_cols = [col for col in num_cols if col not in year_cols]

In [402]:
print(cat_cols)
print('-----------------------')
print(num_cols)

print(len(cat_cols))

print(len(num_cols))

cat_cols.append('Neighborhood')
cat_cols

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'CentralAir', 'Electrical', 'GarageType', 'PavedDrive', 'MiscFeature', 'SaleType', 'SaleCondition', 'YearRemodAdd', 'GarageYrBlt', 'YearBuilt', 'YrSold']
-----------------------
['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'TotRmsAbvGrd', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'Fireplaces', 'GarageCars', 'OverallCond', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'KitchenQual', 'Fu

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'Foundation',
 'Heating',
 'CentralAir',
 'Electrical',
 'GarageType',
 'PavedDrive',
 'MiscFeature',
 'SaleType',
 'SaleCondition',
 'YearRemodAdd',
 'GarageYrBlt',
 'YearBuilt',
 'YrSold',
 'Neighborhood']

## Dealing with Missing Values and Encoding

In [403]:
missing_columns = missing_values_table(df_copy, True)
missing_columns

              n_miss  ratio
MasVnrType      1766 60.500
LotFrontage      486 16.650
GarageYrBlt      160  5.480
MasVnrArea        23  0.790
MSZoning           4  0.140
BsmtFullBath       2  0.070
Utilities          2  0.070
Functional         2  0.070
BsmtHalfBath       2  0.070
GarageArea         1  0.030
GarageCars         1  0.030
KitchenQual        1  0.030
TotalBsmtSF        1  0.030
Electrical         1  0.030
BsmtUnfSF          1  0.030
BsmtFinSF2         1  0.030
BsmtFinSF1         1  0.030
Exterior2nd        1  0.030
Exterior1st        1  0.030
SaleType           1  0.030


['MSZoning',
 'LotFrontage',
 'Utilities',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Electrical',
 'BsmtFullBath',
 'BsmtHalfBath',
 'KitchenQual',
 'Functional',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'SaleType']

In [404]:
rare_analyser(df_copy, cat_cols)

MSZoning : 5
          COUNT  RATIO
MSZoning              
RL         2265  0.776
RM          460  0.158
FV          139  0.048
RH           26  0.009
C (all)      25  0.009


Street : 2
        COUNT  RATIO
Street              
Pave     2907  0.996
Grvl       12  0.004


Alley : 3
            COUNT  RATIO
Alley                   
no_feature   2721  0.932
Grvl          120  0.041
Pave           78  0.027


LotShape : 4
          COUNT  RATIO
LotShape              
Reg        1859  0.637
IR1         968  0.332
IR2          76  0.026
IR3          16  0.005


LandContour : 4
             COUNT  RATIO
LandContour              
Lvl           2622  0.898
HLS            120  0.041
Bnk            117  0.040
Low             60  0.021


Utilities : 2
           COUNT  RATIO
Utilities              
AllPub      2916  0.999
NoSeWa         1  0.000


LotConfig : 5
           COUNT  RATIO
LotConfig              
Inside      2133  0.731
Corner       511  0.175
CulDSac      176  0.060
FR2           85 

In [405]:
df_copy = rare_encoder(df_copy, 0.01)

In [406]:
df_copy = one_hot_encoder(df_copy, cat_cols)

In [407]:
# değişkenlerin standartlatırılması
scaler = MinMaxScaler()
df_copy = pd.DataFrame(scaler.fit_transform(df_copy), columns=df_copy.columns)
df_copy.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingQC,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscVal,MoSold,MSZoning_RL,MSZoning_RM,MSZoning_Rare,Street_Rare,Alley_Pave,Alley_no_feature,LotShape_IR2,LotShape_Rare,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_Rare,LotConfig_CulDSac,LotConfig_FR2,LotConfig_Inside,LotConfig_Rare,LandSlope_Mod,LandSlope_Rare,Condition1_Feedr,Condition1_Norm,Condition1_PosN,Condition1_RRAn,Condition1_Rare,Condition2_Rare,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1Story,HouseStyle_2Story,HouseStyle_Rare,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Hip,RoofStyle_Rare,RoofMatl_Rare,Exterior1st_BrkFace,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Rare,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_BrkFace,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_Rare,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_Rare,MasVnrType_Stone,Foundation_CBlock,Foundation_PConc,Foundation_Rare,Foundation_Slab,Heating_Rare,CentralAir_Y,Electrical_FuseF,Electrical_Rare,Electrical_SBrkr,GarageType_Basment,GarageType_BuiltIn,GarageType_Detchd,GarageType_Rare,GarageType_no_feature,PavedDrive_P,PavedDrive_Y,MiscFeature_Shed,MiscFeature_no_feature,SaleType_New,SaleType_Rare,SaleType_WD,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_Rare,YearRemodAdd_1901-1945,YearRemodAdd_1946-1970,YearRemodAdd_1971-1990,YearRemodAdd_1991-2000,YearRemodAdd_2001-2010,GarageYrBlt_1901-1945,GarageYrBlt_1946-1970,GarageYrBlt_1971-1990,GarageYrBlt_1991-2000,GarageYrBlt_2001-2010,YearBuilt_1901-1945,YearBuilt_1946-1970,YearBuilt_1971-1990,YearBuilt_1991-2000,YearBuilt_2001-2010,YrSold_1901-1945,YrSold_1946-1970,YrSold_1971-1990,YrSold_1991-2000,YrSold_2001-2010,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_Rare,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber
0,0.235,0.222,0.194,0.667,0.500,0.168,0.667,0.429,0.800,0.714,0.250,1.000,0.222,0.167,0.000,0.064,0.228,1.000,0.161,0.414,0.000,0.303,0.400,0.000,0.571,0.500,0.429,0.000,0.667,0.462,1.000,0.000,0.000,0.667,0.400,0.368,0.600,0.600,0.000,0.133,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.091,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,1.000,0.000,0.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,0.000,0.297,0.226,0.556,0.875,0.000,0.333,0.429,0.800,0.714,1.000,0.833,0.307,0.167,0.000,0.122,0.336,1.000,0.286,0.000,0.000,0.205,0.000,0.500,0.571,0.000,0.429,0.000,0.333,0.308,1.00

In [408]:
imputer = KNNImputer(n_neighbors=5)
df_copy = pd.DataFrame(imputer.fit_transform(df_copy), columns=df_copy.columns)
df_copy.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingQC,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscVal,MoSold,MSZoning_RL,MSZoning_RM,MSZoning_Rare,Street_Rare,Alley_Pave,Alley_no_feature,LotShape_IR2,LotShape_Rare,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_Rare,LotConfig_CulDSac,LotConfig_FR2,LotConfig_Inside,LotConfig_Rare,LandSlope_Mod,LandSlope_Rare,Condition1_Feedr,Condition1_Norm,Condition1_PosN,Condition1_RRAn,Condition1_Rare,Condition2_Rare,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1Story,HouseStyle_2Story,HouseStyle_Rare,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Hip,RoofStyle_Rare,RoofMatl_Rare,Exterior1st_BrkFace,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Rare,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_BrkFace,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_Rare,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_Rare,MasVnrType_Stone,Foundation_CBlock,Foundation_PConc,Foundation_Rare,Foundation_Slab,Heating_Rare,CentralAir_Y,Electrical_FuseF,Electrical_Rare,Electrical_SBrkr,GarageType_Basment,GarageType_BuiltIn,GarageType_Detchd,GarageType_Rare,GarageType_no_feature,PavedDrive_P,PavedDrive_Y,MiscFeature_Shed,MiscFeature_no_feature,SaleType_New,SaleType_Rare,SaleType_WD,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_Rare,YearRemodAdd_1901-1945,YearRemodAdd_1946-1970,YearRemodAdd_1971-1990,YearRemodAdd_1991-2000,YearRemodAdd_2001-2010,GarageYrBlt_1901-1945,GarageYrBlt_1946-1970,GarageYrBlt_1971-1990,GarageYrBlt_1991-2000,GarageYrBlt_2001-2010,YearBuilt_1901-1945,YearBuilt_1946-1970,YearBuilt_1971-1990,YearBuilt_1991-2000,YearBuilt_2001-2010,YrSold_1901-1945,YrSold_1946-1970,YrSold_1971-1990,YrSold_1991-2000,YrSold_2001-2010,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_Rare,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber
0,0.235,0.222,0.194,0.667,0.500,0.168,0.667,0.429,0.800,0.714,0.250,1.000,0.222,0.167,0.000,0.064,0.228,1.000,0.161,0.414,0.000,0.303,0.400,0.000,0.571,0.500,0.429,0.000,0.667,0.462,1.000,0.000,0.000,0.667,0.400,0.368,0.600,0.600,0.000,0.133,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.091,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,1.000,0.000,0.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,0.000,0.297,0.226,0.556,0.875,0.000,0.333,0.429,0.800,0.714,1.000,0.833,0.307,0.167,0.000,0.122,0.336,1.000,0.286,0.000,0.000,0.205,0.000,0.500,0.571,0.000,0.429,0.000,0.333,0.308,1.00

In [409]:
df_copy = pd.DataFrame(scaler.inverse_transform(df_copy), columns=df_copy.columns)

In [410]:
df_copy.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingQC,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscVal,MoSold,MSZoning_RL,MSZoning_RM,MSZoning_Rare,Street_Rare,Alley_Pave,Alley_no_feature,LotShape_IR2,LotShape_Rare,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_Rare,LotConfig_CulDSac,LotConfig_FR2,LotConfig_Inside,LotConfig_Rare,LandSlope_Mod,LandSlope_Rare,Condition1_Feedr,Condition1_Norm,Condition1_PosN,Condition1_RRAn,Condition1_Rare,Condition2_Rare,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1Story,HouseStyle_2Story,HouseStyle_Rare,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Hip,RoofStyle_Rare,RoofMatl_Rare,Exterior1st_BrkFace,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Rare,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_BrkFace,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_Rare,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_Rare,MasVnrType_Stone,Foundation_CBlock,Foundation_PConc,Foundation_Rare,Foundation_Slab,Heating_Rare,CentralAir_Y,Electrical_FuseF,Electrical_Rare,Electrical_SBrkr,GarageType_Basment,GarageType_BuiltIn,GarageType_Detchd,GarageType_Rare,GarageType_no_feature,PavedDrive_P,PavedDrive_Y,MiscFeature_Shed,MiscFeature_no_feature,SaleType_New,SaleType_Rare,SaleType_WD,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_Rare,YearRemodAdd_1901-1945,YearRemodAdd_1946-1970,YearRemodAdd_1971-1990,YearRemodAdd_1991-2000,YearRemodAdd_2001-2010,GarageYrBlt_1901-1945,GarageYrBlt_1946-1970,GarageYrBlt_1971-1990,GarageYrBlt_1991-2000,GarageYrBlt_2001-2010,YearBuilt_1901-1945,YearBuilt_1946-1970,YearBuilt_1971-1990,YearBuilt_1991-2000,YearBuilt_2001-2010,YrSold_1901-1945,YrSold_1946-1970,YrSold_1971-1990,YrSold_1991-2000,YrSold_2001-2010,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_Rare,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber
0,60.000,65.000,8450.000,7.000,5.000,196.000,4.000,3.000,4.000,3.000,1.000,6.000,706.000,1.000,0.000,150.000,856.000,5.000,856.000,854.000,0.000,1710.000,1.000,0.000,2.000,1.000,3.000,1.000,4.000,8.000,8.000,0.000,0.000,2.000,2.000,548.000,3.000,3.000,0.000,61.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,1.000,0.000,0.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,20.000,80.000,9600.000,6.000,8.000,0.000,3.000,3.000,4.000,3.000,4.000,5.000,978.000,1.000,0.000,284.000,1262.000,5.000,1262.000,0.000,0.000,1262.000,0.000,1.000

In [411]:
missing_values_table(df_copy)

Empty DataFrame
Columns: [n_miss, ratio]
Index: []


In [412]:
df_copy["NEW_1st*GrLiv"] = df_copy["1stFlrSF"] * df_copy["GrLivArea"]

df_copy["NEW_Garage*GrLiv"] = (df_copy["GarageArea"] * df_copy["GrLivArea"])

df_copy["TotalQual"] = df_copy[["OverallQual", "OverallCond", "ExterQual", "ExterCond", "BsmtCond", "BsmtFinType1",
                      "BsmtFinType2", "HeatingQC", "KitchenQual", "Functional", "FireplaceQu", "GarageQual", "GarageCond", "Fence"]].sum(axis = 1) # 42


# Total Floor
df_copy["NEW_TotalFlrSF"] = df_copy["1stFlrSF"] + df_copy["2ndFlrSF"] # 32

# Total Finished Basement Area
df_copy["NEW_TotalBsmtFin"] = df_copy.BsmtFinSF1 + df_copy.BsmtFinSF2 # 56

# Porch Area
df_copy["NEW_PorchArea"] = df_copy.OpenPorchSF + df_copy.EnclosedPorch + df_copy.ScreenPorch + df_copy["3SsnPorch"] + df_copy.WoodDeckSF # 93

# Total House Area
df_copy["NEW_TotalHouseArea"] = df_copy.NEW_TotalFlrSF + df_copy.TotalBsmtSF # 156

df_copy["NEW_TotalSqFeet"] = df_copy.GrLivArea + df_copy.TotalBsmtSF # 35


# Lot Ratio
df_copy["NEW_LotRatio"] = df_copy.GrLivArea / df_copy.LotArea # 64

df_copy["NEW_RatioArea"] = df_copy.NEW_TotalHouseArea / df_copy.LotArea # 57

df_copy["NEW_GarageLotRatio"] = df_copy.GarageArea / df_copy.LotArea # 69

# MasVnrArea
df_copy["NEW_MasVnrRatio"] = df_copy.MasVnrArea / df_copy.NEW_TotalHouseArea # 36

# Dif Area
df_copy["NEW_DifArea"] = (df_copy.LotArea - df_copy["1stFlrSF"] - df_copy.GarageArea - df_copy.NEW_PorchArea - df_copy.WoodDeckSF) # 73


df_copy["NEW_OverallGrade"] = df_copy["OverallQual"] * df_copy["OverallCond"] # 61


#df_copy["NEW_Restoration"] = df_copy.YearRemodAdd - df_copy.YearBuilt # 31

#df_copy["NEW_HouseAge"] = df_copy.YrSold - df_copy.YearBuilt # 73

#df_copy["NEW_RestorationAge"] = df_copy.YrSold - df_copy.YearRemodAdd # 40

#df_copy["NEW_GarageAge"] = df_copy.GarageYrBlt - df_copy.YearBuilt # 17

#df_copy["NEW_GarageRestorationAge"] = np.abs(df_copy.GarageYrBlt - df_copy.YearRemodAdd) # 30

#df_copy["NEW_GarageSold"] = df_copy.YrSold - df_copy.GarageYrBlt # 48



#drop_list = ["Street", "Alley", "LandContour", "Utilities", "LandSlope","Heating", "PoolQC", "MiscFeature","Neighborhood"]

# drop_list'teki değişkenlerin düşürülmesi
#df_copy.drop(drop_list, axis=1, inplace=True)

In [413]:
df_copy.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingQC,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscVal,MoSold,MSZoning_RL,MSZoning_RM,MSZoning_Rare,Street_Rare,Alley_Pave,Alley_no_feature,LotShape_IR2,LotShape_Rare,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_Rare,LotConfig_CulDSac,LotConfig_FR2,LotConfig_Inside,LotConfig_Rare,LandSlope_Mod,LandSlope_Rare,Condition1_Feedr,Condition1_Norm,Condition1_PosN,Condition1_RRAn,Condition1_Rare,Condition2_Rare,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1Story,HouseStyle_2Story,HouseStyle_Rare,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Hip,RoofStyle_Rare,RoofMatl_Rare,Exterior1st_BrkFace,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Rare,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_BrkFace,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_Rare,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_Rare,MasVnrType_Stone,Foundation_CBlock,Foundation_PConc,Foundation_Rare,Foundation_Slab,Heating_Rare,CentralAir_Y,Electrical_FuseF,Electrical_Rare,Electrical_SBrkr,GarageType_Basment,GarageType_BuiltIn,GarageType_Detchd,GarageType_Rare,GarageType_no_feature,PavedDrive_P,PavedDrive_Y,MiscFeature_Shed,MiscFeature_no_feature,SaleType_New,SaleType_Rare,SaleType_WD,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_Rare,YearRemodAdd_1901-1945,YearRemodAdd_1946-1970,YearRemodAdd_1971-1990,YearRemodAdd_1991-2000,YearRemodAdd_2001-2010,GarageYrBlt_1901-1945,GarageYrBlt_1946-1970,GarageYrBlt_1971-1990,GarageYrBlt_1991-2000,GarageYrBlt_2001-2010,YearBuilt_1901-1945,YearBuilt_1946-1970,YearBuilt_1971-1990,YearBuilt_1991-2000,YearBuilt_2001-2010,YrSold_1901-1945,YrSold_1946-1970,YrSold_1971-1990,YrSold_1991-2000,YrSold_2001-2010,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_Rare,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,NEW_1st*GrLiv,NEW_Garage*GrLiv,TotalQual,NEW_TotalFlrSF,NEW_TotalBsmtFin,NEW_PorchArea,NEW_TotalHouseArea,NEW_TotalSqFeet,NEW_LotRatio,NEW_RatioArea,NEW_GarageLotRatio,NEW_MasVnrRatio,NEW_DifArea,NEW_OverallGrade
0,60.000,65.000,8450.000,7.000,5.000,196.000,4.000,3.000,4.000,3.000,1.000,6.000,706.000,1.000,0.000,150.000,856.000,5.000,856.000,854.000,0.000,1710.000,1.000,0.000,2.000,1.000,3.000,1.000,4.000,8.000,8.000,0.000,0.000,2.000,2.000,548.000,3.000,3.000,0.000,61.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,1.000,0.000,0.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00

In [414]:
split_index = 1460

# İlk parça: 0'dan split_index'e kadar
preprocessed_train = df_copy.iloc[:split_index]

# İkinci parça: split_index'ten sona kadar
preprocessed_test = df_copy.iloc[split_index:]

In [415]:
preprocessed_train['SalePrice'] = df_train['SalePrice'].copy()
preprocessed_train['SalePrice'].head()

C:\Users\BORA\AppData\Local\Temp\ipykernel_12188\4290186271.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preprocessed_train['SalePrice'] = df_train['SalePrice'].copy()


0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

## TO CSV

In [416]:
preprocessed_train.to_csv('train_preprocessed.csv', index = False)
preprocessed_test.to_csv('test_preprocessed.csv', index = False)

## Categorization of Year Features

In [417]:
'''
df_copy['YearBuilt'].describe()
sns.set(rc = {'figure.figsize' : (20,8)})
sns.histplot(df_copy['YearBuilt'], kde=True, binrange=((1872, 2010)), bins = 139);
df_copy['GarageYrBlt'].describe()
sns.set(rc = {'figure.figsize' : (20,8)})
sns.histplot(df_copy['GarageYrBlt'], kde=True, binrange=((1895, 2010)), bins = 116);
df_copy['YearRemodAdd'].describe()
sns.set(rc = {'figure.figsize' : (20,8)})
sns.histplot(df_copy['YearRemodAdd'], kde=True, binrange=((1950, 2010)), bins = 61);

'''

"\ndf_copy['YearBuilt'].describe()\nsns.set(rc = {'figure.figsize' : (20,8)})\nsns.histplot(df_copy['YearBuilt'], kde=True, binrange=((1872, 2010)), bins = 139);\ndf_copy['GarageYrBlt'].describe()\nsns.set(rc = {'figure.figsize' : (20,8)})\nsns.histplot(df_copy['GarageYrBlt'], kde=True, binrange=((1895, 2010)), bins = 116);\ndf_copy['YearRemodAdd'].describe()\nsns.set(rc = {'figure.figsize' : (20,8)})\nsns.histplot(df_copy['YearRemodAdd'], kde=True, binrange=((1950, 2010)), bins = 61);\n\n"